In [1]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [2]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions
from debiaswe.data import load_gender_seed

import torch

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [3]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')
print(E)
words = E.words
print("Words:", len(words))

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Words: 26423
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [4]:
# gender direction
v_gender = E.diff('she', 'he')

# Uncomment below for direction based on multiple definitional pairs
# with open('./data/definitional_pairs.json', "r") as f:
#     defs = json.load(f)
# v_gender = we.doPCA(defs, E).components_[0]

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [5]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender, thresh=1)

# for (a,b,c) in a_gender:
#     print(a+"-"+b)
we.viz(a_gender)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
(135018, 300)
   0                          she | he                           1.0
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13                

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [6]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.23798442, 'maestro'),
  (-0.21665451, 'statesman'),
  (-0.20758669, 'skipper'),
  (-0.20267202, 'protege'),
  (-0.2020676, 'businessman'),
  (-0.19492392, 'sportsman'),
  (-0.18836352, 'philosopher'),
  (-0.1807366, 'marksman'),
  (-0.1728986, 'captain'),
  (-0.16785555, 'architect'),
  (-0.16702037, 'financier'),
  (-0.16313636, 'warrior'),
  (-0.15280862, 'major_leaguer'),
  (-0.15001445, 'trumpeter'),
  (-0.14718868, 'broadcaster'),
  (-0.14637242, 'magician'),
  (-0.14401694, 'fighter_pilot'),
  (-0.13782285, 'boss'),
  (-0.137182, 'industrialist'),
  (-0.13684885, 'pundit')],
 [(0.19714224, 'interior_designer'),
  (0.20833439, 'housekeeper'),
  (0.21560375, 'stylist'),
  (0.2236317, 'bookkeeper'),
  (0.23776126, 'maid'),
  (0.24125955, 'nun'),
  (0.24782579, 'nanny'),
  (0.24929334, 'hairdresser'),
  (0.24946158, 'paralegal'),
  (0.25276464, 'ballerina'),
  (0.25718823, 'socialite'),
  (0.26647124, 'librarian'),
  (0.27317622, 'receptionist'),
  (0.27540293, 'waitress'),
  (

## Find soft debias transform using SGD
This code is largely based on code from https://github.com/TManzini/DebiasMulticlassWordEmbedding.

In [7]:
from debiaswe.debias import soft_debias

In [8]:
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)

In [9]:
soft_debias(E, gender_specific_words, defs)

Loss @ Epoch #0: 873031.625
Loss @ Epoch #10: 591185.3125
Loss @ Epoch #20: 492240.09375
Loss @ Epoch #30: 420856.03125
Loss @ Epoch #40: 364923.78125
Loss @ Epoch #50: 319460.03125
Loss @ Epoch #60: 281638.4375
Loss @ Epoch #70: 249621.390625
Loss @ Epoch #80: 222164.140625
Loss @ Epoch #90: 198385.859375
Loss @ Epoch #100: 177639.28125
Loss @ Epoch #110: 159427.75
Loss @ Epoch #120: 143365.125
Loss @ Epoch #130: 129142.140625
Loss @ Epoch #140: 116504.296875
Loss @ Epoch #150: 105241.03125
Loss @ Epoch #160: 95176.4765625
Loss @ Epoch #170: 86161.25
Loss @ Epoch #180: 78068.265625
Loss @ Epoch #190: 70789.5390625
Loss @ Epoch #200: 64231.1484375
Loss @ Epoch #210: 58312.2421875
Loss @ Epoch #220: 52963.37109375
Loss @ Epoch #230: 48123.00390625
Loss @ Epoch #240: 43738.140625
Loss @ Epoch #250: 39761.63671875
Loss @ Epoch #260: 36151.9765625
Loss @ Epoch #270: 32873.0703125
Loss @ Epoch #280: 29892.013671875
Loss @ Epoch #290: 27180.232421875
Loss @ Epoch #300: 24711.841796875
Loss @

In [10]:
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)
we.viz(a_gender_debiased)

   0                          she | he                           1.0
   1                      herself | himself                      0.92
   2                          her | his                          0.90
   3                        woman | man                          0.75
   4                     daughter | son                          0.67
   5                businesswoman | businessman                  0.65
   6                         girl | boy                          0.65
   7                      actress | actor                        0.65
   8                   chairwoman | chairman                     0.63
   9                      heroine | hero                         0.62
  10                       mother | father                       0.60
  11                  spokeswoman | spokesman                    0.59
  12                       sister | brother                      0.59
  13                        girls | boys                         0.59
  14                 

In [11]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.21934132, 'hitman'),
  (-0.19997133, 'warrior'),
  (-0.19754022, 'baron'),
  (-0.19515152, 'philosopher'),
  (-0.18315929, 'choreographer'),
  (-0.18254068, 'assassin'),
  (-0.16508263, 'marshal'),
  (-0.16474639, 'critic'),
  (-0.16374741, 'statesman'),
  (-0.16007188, 'marksman'),
  (-0.15895528, 'sailor'),
  (-0.15825412, 'tycoon'),
  (-0.15640043, 'serviceman'),
  (-0.15398164, 'boxer'),
  (-0.15176047, 'mobster'),
  (-0.14897011, 'therapist'),
  (-0.14655036, 'historian'),
  (-0.14589757, 'alter_ego'),
  (-0.14529522, 'soldier'),
  (-0.14337139, 'physicist')],
 [(-0.0019266795, 'bookkeeper'),
  (-0.0012663512, 'infielder'),
  (-0.000324311, 'manager'),
  (0.0017308401, 'solicitor'),
  (0.0025848814, 'stockbroker'),
  (0.006106122, 'chef'),
  (0.006671122, 'shopkeeper'),
  (0.0072377175, 'landlord'),
  (0.010533481, 'freelance_writer'),
  (0.01263684, 'illustrator'),
  (0.012786401, 'proprietor'),
  (0.01373632, 'registered_nurse'),
  (0.0153875975, 'associate_dean'),
  (0.02